In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent))

In [9]:
from itertools import chain

from front import SnakeGameWindow
from agent import PlayerSnakeAgent, AStarSnakeAgent, AStarOffensiveSnakeAgent
from world import SnakeWorld, EuclidianDistanceHeuristic, ManhattanDistanceHeuristic

TODO: explication des règles du jeu

# Adversaire passif

In [ ]:
width = 21
height = 21

world = SnakeWorld(width, height, n_food=1)

blue = PlayerSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1)
)

yellow = AStarSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
)

player_agents = [blue]
ai_agents = [yellow]

for agent in chain(player_agents, ai_agents):
    world.attach_agent(agent)

gui = SnakeGameWindow(
    world,
    player_agents=player_agents,
    ai_agents=ai_agents,
    time_step=100,
    ui_size_coeff=20
)
gui.title("Adversaire passif")
gui.mainloop()

TODO: explication de l'algorithme de l'IA passive:
- essaie de grandir le plus vite possible donc se dirige toujours vers la nourriture la plus proche
- s'adapte aux mouvements du joueurs en recalculant sont chemin à chaque pas du jeu

# Adversaire offensif

In [ ]:
width = 21
height = 21

world = SnakeWorld(width, height, n_food=1)

blue = PlayerSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1)
)

yellow = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
)

yellow.add_opponent(blue)

player_agents = [blue]
ai_agents = [yellow]

for agent in chain(player_agents, ai_agents):
    world.attach_agent(agent)

gui = SnakeGameWindow(
    world,
    player_agents=player_agents,
    ai_agents=ai_agents,
    time_step=100,
    ui_size_coeff=20
)
gui.title("Adversaire offensif")
gui.mainloop()

TODO: explication de l'algorithme de l'IA offensive (avec des ptites formules de maths)

# Paramétrer le comportement des IA

TODO: explication des deux paramètres:
- latency pour dégrader les performances de l'IA et la rendre possible à piéger
- caution pour rendre une IA plus ou moins prudente

In [ ]:
width = 30
height = 30
world = SnakeWorld(width, height, n_food=2)

blue = PlayerSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1)
)

yellow = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=1
)

purple = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=2
)

green = AStarSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=3
)

yellow.add_opponent(purple)
yellow.add_opponent(blue)
purple.add_opponent(blue)

player_agents = [blue]
ai_agents = [yellow, purple, green]

for agent in chain(player_agents, ai_agents):
    world.attach_agent(agent)

gui = SnakeGameWindow(
    world,
    player_agents=player_agents,
    ai_agents=ai_agents,
    time_step=100,
    ui_size_coeff=20
)
gui.title("Comportement paramétrable")
gui.mainloop()

# Combat de programmes

Avec la musique de TRON, c'est mieux !

In [ ]:
try:
    import pygame
    class SnakeGameWindowWithCoolMusic(SnakeGameWindow):
        def mainloop(self, n = 0):
            pygame.mixer.init()
            pygame.mixer.music.load("music/tron.mp3")
            pygame.mixer.music.play(-1)
            pygame.mixer.music.set_pos(9.7)
            return super().mainloop(n)

        def destroy(self):
            pygame.mixer.music.stop()
            return super().destroy()
except ModuleNotFoundError:
    SnakeGameWindowWithCoolMusic = SnakeGameWindow

Scénario:
- Les jaunes attaquent les bleus
- Les bleus attaquent les rouges
- Les rouges attaquent les verts
- Les verts attaquent les jaunes

In [ ]:
width = 30
height = 30
world = SnakeWorld(width, height, n_food=3)

blue = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=0
)

dark_blue = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(9, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=0
)


yellow = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=1
)

orange = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-10, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=1
)


purple = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=4,
    caution=4
)

red = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(9, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=4,
    caution=4
)


green = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=3
)

light_green = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-10, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=3
)

blue.add_opponent(purple)
blue.add_opponent(red)
dark_blue.add_opponent(purple)
dark_blue.add_opponent(red)

purple.add_opponent(green)
purple.add_opponent(light_green)
red.add_opponent(green)
red.add_opponent(light_green)

green.add_opponent(yellow)
green.add_opponent(orange)
light_green.add_opponent(yellow)
light_green.add_opponent(orange)

yellow.add_opponent(blue)
yellow.add_opponent(dark_blue)
orange.add_opponent(blue)
orange.add_opponent(dark_blue)


player_agents = []
ai_agents = [blue, yellow, purple, green, red, orange, light_green, dark_blue]

for agent in chain(player_agents, ai_agents):
    world.attach_agent(agent)

gui = SnakeGameWindowWithCoolMusic(
    world,
    player_agents=player_agents,
    ai_agents=ai_agents,
    time_step=50,
        explain_ai=True,
    ui_size_coeff=20
)
gui.title("Combat de programmes")
gui.mainloop()